In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
%matplotlib inline

In [ ]:
df = pd.read_excel('/content/besprovodnaya_svyaz_classification.xlsx')
df

,area,label,keywords_eng,pub_date,link,sentence,probability
0,WAN,5GS,"""5GS"" OR ""5G System""",2019-12-31,https://seekingalpha.com/article/4314620-apple...,SummaryI am increasing my price targets for Ap...,0.942507
1,WAN,5GS,"""5GS"" OR ""5G System""",2019-12-31,http://www.govinfosecurity.com/huawei-us-trade...,Financial Outlook Despite the concerns and tra...,0.980529
2,WAN,5GS,"""5GS"" OR ""5G System""",2019-12-31,https://www.financialexpress.com/industry/tech...,"Also, Huawei said on Tuesday that its annual r...",0.949668
3,WAN,5GS,"""5GS"" OR ""5G System""",2019-12-31,https://telecoms.com/501552/globaldata-expect-...,Device manufacturers will not go 100% unless t...,0.900437
4,WAN,5GS,"""5GS"" OR ""5G System""",2019-12-31,https://www.financialexpress.com/industry/huaw...,Chairman Eric Xu said Huawei’s sales revenue f...,0.974104
...,...,...,...,...,...,...,...
110480,Спутниковые технологии связи,Решения на основе технологий спутникового инте...,"(""Internet of Things"" OR ""IoT"") AND ""Satellite""",2011-02-16,https://readwrite.com/2011/02/16/mobile-phones...,"By 2014, there will be greater than 400 millio...",0.937829
110481,Спутниковые технологии связи,Решения на основе технологий спутникового инте...,"(""Internet of Things"" OR ""IoT"") AND ""Satellite""",2010-09-24,https://www.itnews.com/article/2414852/the-int...,"Today, about 35 billion devices connect to the...",0.947755
110482,Спутниковые технологии связи,Решения на основе технологий спутникового инте...,"(""Internet of Things"" OR ""IoT"") AND ""Satellite""",2010-09-24,https://www.itnews.com/article/2219590/how-cio...,"Making New Friends Today, about 35 billion dev...",0.936742
110483,Спутниковые технологии связи,Решения на основе технологий спутникового инте...,"(""Internet of Things"" OR ""IoT"") AND ""Satellite""",2010-09-22,https://gigaom.com/2010/09/22/gigaoms-top-15-m...,"In 2001, the company had sales of £146.3 milli...",0.909740


In [ ]:
df['sentence'].head()

0    SummaryI am increasing my price targets for Ap...
1    Financial Outlook Despite the concerns and tra...
2    Also, Huawei said on Tuesday that its annual r...
3    Device manufacturers will not go 100% unless t...
4    Chairman Eric Xu said Huawei’s sales revenue f...
Name: sentence, dtype: object

In [ ]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 123 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 51.2 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 8.7 MB 58.2 MB/s 
     |████████████████████████████████| 138 kB 55.1 MB/s 
     |████████████████████████████████| 127 kB 52.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


100%|██████████| 231508/231508 [00:00<00:00, 3094296.29B/s]


In [ ]:
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

['[CLS]', 'here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.', '[SEP]']


In [ ]:
list(tokenizer.vocab.keys())[5000:5020]

['knight',
 'lap',
 'survey',
 'ma',
 '##ow',
 'noise',
 'billy',
 '##ium',
 'shooting',
 'guide',
 'bedroom',
 'priest',
 'resistance',
 'motor',
 'homes',
 'sounded',
 'giant',
 '##mer',
 '150',
 'scenes']

In [ ]:
# Define a new example sentence with multiple meanings of the word "bank"
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102


In [ ]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


100%|██████████| 407873900/407873900 [00:13<00:00, 31345957.46B/s]


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [ ]:
!pip install bert-serving-server bert-serving-client

     |████████████████████████████████| 61 kB 199 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=08fe4c0f1139a8e5f822d098e3071457a9bea403ad76b2cb7ccbc23e3e65c7a6
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Python program to generate word vectors using Word2Vec

# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec

#  Reads ‘alice.txt’ file
sample = open("/content/asd.txt")
s = sample.read()
 
# Replaces escape character with space
f = s.replace("\n", " ")
 
data = []
 
# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []
     
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
 
    data.append(temp)

print(data)

# # Create CBOW model
# model1 = gensim.models.Word2Vec(data, min_count = 1, max_vocab_size = 100, window = 5)

# # Print results
# print("Cosine similarity between 'self' " +
#                "and 'catering' - CBOW : ",
#     model1.wv.similarity('self', 'catering'))
     
# print("Cosine similarity between 'any' " +
#                  "and 'job' - CBOW : ",
#       model1.wv.similarity('any', 'job'))
 
# # Create Skip Gram model
# model2 = gensim.models.Word2Vec(data, min_count = 1, max_vocab_size = 100, window = 5, sg = 1)
 
# # Print results
# print("Cosine similarity between 'self' " +
#           "and 'catering' - Skip Gram : ",
#     model2.wv.similarity('self', 'catering'))
     
# print("Cosine similarity between 'any' " +
#             "and 'job' - Skip Gram : ",
#       model2.wv.similarity('any', 'job'))

[['self', 'catering', 'is', 'a', 'perspective', 'service', 'in', 'our', 'century', ',', 'where', 'any', 'job', 'is', 'not', 'relevant', '.']]


In [ ]:
# echo-server.py

import socket

HOST = "127.0.0.1"  # Standard loopback interface address (localhost)
PORT = 65432  # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            data = conn.recv(1024)
            if not data:
                break
            conn.sendall(data)

In [ ]:
df['sentence'].to_csv('pandas.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
df.iloc[:10000, 5].to_csv('pandas.txt', header=None, index=None, sep=' ', mode='a')

In [ ]:
# Python program to generate word vectors using Word2Vec
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

warnings.filterwarnings(action = 'ignore')

import gensim
from gensim.models import Word2Vec

#  Reads ‘alice.txt’ file
sample = open("/content/pandas.txt")
s = sample.read()
 
# Replaces escape character with space
f = s.replace("\n", " ")
 
data = []
 
# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []
     
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
 
    data.append(temp)

print(data)

[['``', 'summaryi', 'am', 'increasing', 'my', 'price', 'targets', 'for', 'apple', 'to', '$', '325', 'per', 'share', 'by', 'may', 'of', '2020', 'and', '$', '400', 'per', 'share', 'longer', 'term', 'by', 'may', '2021.wearables', ',', 'home', 'and', 'accessories', 'grew', 'by', '90.88', '%', 'and', 'services', 'grew', 'by', '41.56', '%', 'over', 'the', 'past', 'two', 'fiscal', 'years', 'and', 'these', 'business', 'segments', 'will', 'be', 'critical', 'to', 'apple', "'s", 'future', 'growth', '.', "''"], ['``', 'financial', 'outlook', 'despite', 'the', 'concerns', 'and', 'trade', 'ban', ',', 'xu', "'s", 'message', 'played', 'up', 'huawei', "'s", 'successes', 'over', 'the', 'past', 'year', ',', 'which', 'include', 'estimated', 'sales', 'revenue', 'of', 'more', 'than', '850', 'billion', 'yuan', '(', '$', '122', 'billion', ')', ',', 'an', '18', 'percent', 'year-over', 'year', 'increase', '.', "''"], ['``', 'also', ',', 'huawei', 'said', 'on', 'tuesday', 'that', 'its', 'annual', 'revenue', 'for

In [ ]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, max_vocab_size = 10000, window = 5)

# Print results
print("Cosine similarity between 'increasing' " +
               "and 'risen' - CBOW : ",
    model1.wv.similarity('increasing', 'risen'))
     
print("Cosine similarity between 'market' " +
                 "and 'financial' - CBOW : ",
      model1.wv.similarity('market', 'financial'))
 
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, max_vocab_size = 100, window = 5, sg = 1)
 
# Print results
print("Cosine similarity between 'increasing' " +
          "and 'risen' - Skip Gram : ",
    model2.wv.similarity('increasing', 'risen'))
     
print("Cosine similarity between 'market' " +
            "and 'financial' - Skip Gram : ",
      model2.wv.similarity('market', 'financial'))

Cosine similarity between 'increasing' and 'risen' - CBOW :  0.67660147
Cosine similarity between 'market' and 'financial' - CBOW :  0.42015877


KeyError: ignored

In [ ]:
from nltk.corpus import stopwords
from string import ascii_lowercase
import pandas as pd
import gensim, os, re, itertools, nltk, snowballstemmer
from collections import Counter

# initialize stemmer
stemmer = snowballstemmer.EnglishStemmer()

# grab stopword list, extend it a bit, and then turn it into a set for later
stop = stopwords.words('english')
data = df
# remove characters and stoplist words, then generate dictionary of unique words
data['sentence'].replace('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', inplace=True, regex=True)
wordlist = filter(None, " ".join(list(set(list(itertools.chain(*data['sentence'].str.split(' ')))))).split(" "))
data['stemmed_text_data'] = [' '.join(filter(None,filter(lambda word: word not in stop, line))) for line in data['sentence'].str.lower().str.split(' ')]

# remove all words that don't occur at least 5 times and then stem the resulting docs
minimum_count = 5
str_frequencies = pd.DataFrame(list(Counter(filter(None,list(itertools.chain(*data['stemmed_text_data'].str.split(' '))))).items()),columns=['word','count'])
low_frequency_words = set(str_frequencies[str_frequencies['count'] < minimum_count]['word'])
# data['stemmed_text_data'] = [' '.join(filter(None,filter(lambda word: word not in low_frequency_words, line))) for line in data['stemmed_text_data'].str.split(' ')]
# data['stemmed_text_data'] = [" ".join(stemmer.stemWords(re.sub('[!"#%\'()*+,-./:;<=>?@\[\]^_`{|}~1234567890’”“′‘\\\]',' ', next_text).split(' '))) for next_text in data['stemmed_text_data']]

# # run word2vec model and then save it
texts_stemmed = filter(None, [next_text.strip(' ').split(' ') for next_text in data['sentence']])
# texts_stemmed = filter(None, [next_text.strip(' ').split(' ') for next_text in data['stemmed_text_data']])
w2vmodel_stemmed = gensim.models.Word2Vec(texts_stemmed, size=100, window=5, min_count=5, workers=4)
w2vmodel_stemmed.save('w2v_stemmed_model')

In [ ]:
w2vmodel_stemmed.wv.vocab

In [ ]:
print("Cosine similarity between 'increasing' " +
          "and 'risen' - Skip Gram : ",
    w2vmodel_stemmed.wv.similarity('increase', 'rise'))
     
print("Cosine similarity between 'market' " +
            "and 'financial' - Skip Gram : ",
      w2vmodel_stemmed.wv.similarity('market', 'finance'))

KeyError: ignored

In [ ]:
df.iloc[:10000, 5].to_csv('first.csv')